In [39]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, confusion_matrix, classification_report,accuracy_score, f1_score
import numpy as np
import pandas as pd
import random
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve , auc
from sklearn.metrics.cluster import fowlkes_mallows_score as gmean
import statistics as stat

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
df=pd.read_csv('/content/drive/MyDrive/ecoli-0-1-3-7_vs_2-6.csv')
df

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Class
0,49.0,29.0,48.0,5.0,56.0,24.0,35.0,negative
1,7.0,4.0,48.0,5.0,54.0,35.0,44.0,negative
2,56.0,4.0,48.0,5.0,49.0,37.0,46.0,negative
3,59.0,49.0,48.0,5.0,52.0,45.0,36.0,negative
4,23.0,32.0,48.0,5.0,55.0,25.0,35.0,negative
...,...,...,...,...,...,...,...,...
276,77.0,57.0,1.0,5.0,37.0,54.0,1.0,positive
277,66.0,49.0,1.0,5.0,54.0,56.0,36.0,positive
278,71.0,46.0,1.0,5.0,52.0,59.0,3.0,positive
279,67.0,55.0,1.0,5.0,66.0,58.0,16.0,positive


In [42]:
df["Class"].value_counts()
class_label=df['Class']
class_label

0      negative
1      negative
2      negative
3      negative
4      negative
         ...   
276    positive
277    positive
278    positive
279    positive
280    positive
Name: Class, Length: 281, dtype: object

In [43]:
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df.head()


,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Class
0,49.0,29.0,48.0,5.0,56.0,24.0,35.0,0
1,7.0,4.0,48.0,5.0,54.0,35.0,44.0,0
2,56.0,4.0,48.0,5.0,49.0,37.0,46.0,0
3,59.0,49.0,48.0,5.0,52.0,45.0,36.0,0
4,23.0,32.0,48.0,5.0,55.0,25.0,35.0,0


In [44]:
minor=df[df["Class"]==1]
major=df[df["Class"]==0]
minor=minor.drop(["Class"], axis=1)
minority=minor.to_numpy()
minor.shape
major.shape
diff=major.shape[0]-minor.shape[0]
r=diff
loop=diff/minor.shape[0]
if(diff%minor.shape[0]==0):
    iter=int(diff/minor.shape[0])
else:
  iter=int(diff/minor.shape[0])+1
minor.shape


(7, 7)

In [45]:
for id in range(r):
  if(diff>0):
    distances = np.linalg.norm(minority - minority[id], axis=1)  #calculating the distances to other instances
    k = 4
    nearest_neighbor_ids = distances.argsort()[:k] #finding 3 nearest neighbours n1,n2 and n3
    n1=minority[nearest_neighbor_ids[1]]
    n2=minority[nearest_neighbor_ids[2]]
    n3=minority[nearest_neighbor_ids[3]]
    r1=random.uniform(0,1)
    r2=random.uniform(0,1)
    w=random.uniform(0.5,1)
    velocity= w+r1*(n1-n3) + r2*(n1-n2)         #velocity vector
    new_minor=minority[id]+velocity             #position update
    d=minor.shape[1] 
    minB=minor.min().to_numpy()
    maxB=minor.max().to_numpy()
    for i in range(d):
      if(new_minor[i]<minB[i]):
        new_minor[i]=minB[i];
      if(new_minor[i]>maxB[i]):
        new_minor[i]=maxB[i];
    minority=np.append(minority,new_minor)
    minority=np.reshape(minority,(-1, minor.shape[1]))
    diff=diff-1

minority.shape

(274, 7)

In [46]:
df_minor = pd.DataFrame(minority, columns =['Mcg','Gvh','Lip','Chg','Aac','Alm1','Alm2'])
df_minor

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2
0,85.0,53.0,48.000000,5.0,53.000000,52.0,35.000
1,63.0,49.0,48.000000,5.0,54.000000,76.0,79.000
2,77.0,57.0,1.000000,5.0,37.000000,54.0,1.000
3,66.0,49.0,1.000000,5.0,54.000000,56.0,36.000
4,71.0,46.0,1.000000,5.0,52.000000,59.0,3.000
...,...,...,...,...,...,...,...
269,85.0,57.0,22.163930,5.0,66.000000,76.0,79.000
270,85.0,57.0,48.000000,5.0,54.043422,76.0,72.471
271,63.0,57.0,48.000000,5.0,66.000000,76.0,79.000
272,85.0,57.0,18.203489,5.0,51.419706,76.0,79.000


In [47]:
dup=[]
for i in range(df_minor.shape[0]):
  dup.append(1)
Class=np.array(dup)
Class=np.reshape(Class,(-1,1))
df_minor_class=pd.DataFrame(Class,columns=['Class'])
df_minor['Class']=df_minor_class
df_minor

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Class
0,85.0,53.0,48.000000,5.0,53.000000,52.0,35.000,1
1,63.0,49.0,48.000000,5.0,54.000000,76.0,79.000,1
2,77.0,57.0,1.000000,5.0,37.000000,54.0,1.000,1
3,66.0,49.0,1.000000,5.0,54.000000,56.0,36.000,1
4,71.0,46.0,1.000000,5.0,52.000000,59.0,3.000,1
...,...,...,...,...,...,...,...,...
269,85.0,57.0,22.163930,5.0,66.000000,76.0,79.000,1
270,85.0,57.0,48.000000,5.0,54.043422,76.0,72.471,1
271,63.0,57.0,48.000000,5.0,66.000000,76.0,79.000,1
272,85.0,57.0,18.203489,5.0,51.419706,76.0,79.000,1


In [48]:
majority=major.to_numpy()
majority=np.reshape(majority,(-1,major.shape[1]))
majority
df_major = pd.DataFrame(majority, columns = ['Mcg','Gvh','Lip','Chg','Aac','Alm1','Alm2','Class'])
df_major

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Class
0,49.0,29.0,48.0,5.0,56.0,24.0,35.0,0.0
1,7.0,4.0,48.0,5.0,54.0,35.0,44.0,0.0
2,56.0,4.0,48.0,5.0,49.0,37.0,46.0,0.0
3,59.0,49.0,48.0,5.0,52.0,45.0,36.0,0.0
4,23.0,32.0,48.0,5.0,55.0,25.0,35.0,0.0
...,...,...,...,...,...,...,...,...
269,74.0,56.0,48.0,5.0,47.0,68.0,3.0,0.0
270,71.0,57.0,48.0,5.0,48.0,35.0,32.0,0.0
271,61.0,6.0,48.0,5.0,44.0,39.0,38.0,0.0
272,59.0,61.0,48.0,5.0,42.0,42.0,37.0,0.0


In [49]:
balanced_df=pd.concat([df_minor,df_major],axis=0,ignore_index=True)
balanced_df

,Mcg,Gvh,Lip,Chg,Aac,Alm1,Alm2,Class
0,85.0,53.0,48.0,5.0,53.0,52.0,35.0,1.0
1,63.0,49.0,48.0,5.0,54.0,76.0,79.0,1.0
2,77.0,57.0,1.0,5.0,37.0,54.0,1.0,1.0
3,66.0,49.0,1.0,5.0,54.0,56.0,36.0,1.0
4,71.0,46.0,1.0,5.0,52.0,59.0,3.0,1.0
...,...,...,...,...,...,...,...,...
543,74.0,56.0,48.0,5.0,47.0,68.0,3.0,0.0
544,71.0,57.0,48.0,5.0,48.0,35.0,32.0,0.0
545,61.0,6.0,48.0,5.0,44.0,39.0,38.0,0.0
546,59.0,61.0,48.0,5.0,42.0,42.0,37.0,0.0


In [50]:

from sklearn.model_selection import StratifiedKFold
kf=StratifiedKFold(n_splits=5)
x_b=balanced_df.drop(['Class'],axis='columns')
y_b=balanced_df.Class




In [51]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
model_b_s=SVC()

model_b_k=KNeighborsClassifier(n_neighbors=3)


model_b_d=DecisionTreeClassifier(random_state=0)

In [52]:
#precision
precision_s=[]
f1_s=[]
recall_s=[]
accuracy_s=[]
Auc_s=[]
Gmean_s=[]

precision_k=[]
f1_k=[]
recall_k=[]
accuracy_k=[]
Auc_k=[]
Gmean_k=[]

precision_d=[]
f1_d=[]
recall_d=[]
accuracy_d=[]
Auc_d=[]
Gmean_d=[]
for train_index_b,test_index_b in kf.split(x_b,y_b):
  x_train_b,x_test_b=x_b.iloc[train_index_b],x_b.iloc[test_index_b]
  y_train_b,y_test_b=y_b.iloc[train_index_b],y_b.iloc[test_index_b]
  model_b_s.fit(x_train_b,y_train_b)
  prediction_b_s = model_b_s.predict(x_test_b)

  model_b_k.fit(x_train_b,y_train_b)
  prediction_b_k = model_b_k.predict(x_test_b)

  model_b_d.fit(x_train_b,y_train_b)
  prediction_b_d = model_b_d.predict(x_test_b)

  precision_s.append(precision_score(y_test_b,prediction_b_s))
  precision_k.append(precision_score(y_test_b,prediction_b_k))
  precision_d.append(precision_score(y_test_b,prediction_b_d))

  recall_s.append(recall_score(y_test_b,prediction_b_s))
  recall_k.append(recall_score(y_test_b,prediction_b_k))
  recall_d.append(recall_score(y_test_b,prediction_b_d))

  f1_s.append(f1_score(y_test_b,prediction_b_s))
  f1_k.append(f1_score(y_test_b,prediction_b_k))
  f1_d.append(f1_score(y_test_b,prediction_b_d))

  accuracy_s.append(accuracy_score(y_test_b,prediction_b_s))
  accuracy_k.append(accuracy_score(y_test_b,prediction_b_k))
  accuracy_d.append(accuracy_score(y_test_b,prediction_b_d))


  s_fpr_b,s_tpr_b,threshold_s_b=roc_curve(y_test_b,prediction_b_s)
  k_fpr_b,k_tpr_b,threshold_k_b=roc_curve(y_test_b,prediction_b_k)
  d_fpr_b,d_tpr_b,threshold_d_b=roc_curve(y_test_b,prediction_b_d)


  Auc_s.append(auc(s_fpr_b,s_tpr_b))
  Auc_k.append(auc(k_fpr_b,k_tpr_b))
  Auc_d.append(auc(d_fpr_b,d_tpr_b))


  Gmean_s.append(gmean(y_test_b,prediction_b_s))
  Gmean_k.append(gmean(y_test_b,prediction_b_k))
  Gmean_d.append(gmean(y_test_b,prediction_b_d))


print("-------SVM--------")
print("Gmean_SVM:",Gmean_s)
print("AUC_SVM:",Auc_s)
print("f1_SVM:",f1_s)
print("precision_SVM:",precision_s)
print("recall_SVM:",recall_s)

print("mean of Gmean_s:",stat.mean(Gmean_s))
print("mean of AUC_s:",stat.mean(Auc_s))
print("mean of f1_s:",stat.mean(f1_s))
print("mean of precision_s:",stat.mean(precision_s))
print("mean of recall_s:",stat.mean(recall_s))

print("standard deviation of Gmean_s:",stat.stdev(Gmean_s))
print("standard deviation of AUC_s:",stat.stdev(Auc_s))
print("standard deviation of f1_s:",stat.stdev(f1_s))
print("standard deviation of precision_s:",stat.stdev(precision_s))
print("standard deviation of recall_s:",stat.stdev(recall_s))

print("\n")
print("-------KNN--------")
print("Gmean_KNN:",Gmean_k)
print("AUC_KNN:",Auc_k)
print("f1_KNN:",f1_k)
print("precision_KNN:",precision_k)
print("recall_KNN:",recall_k)

print("mean of Gmean_k:",stat.mean(Gmean_k))
print("mean of AUC_k:",stat.mean(Auc_k))
print("mean of f1_k:",stat.mean(f1_k))
print("mean of precision_k:",stat.mean(precision_k))
print("mean of recall_k:",stat.mean(recall_k))

print("standard deviation of Gmean_k:",stat.stdev(Gmean_k))
print("standard deviation of AUC_k:",stat.stdev(Auc_k))
print("standard deviation of f1_k:",stat.stdev(f1_k))
print("standard deviation of precision_k:",stat.stdev(precision_k))
print("standard deviation of recall_k:",stat.stdev(recall_k))

print("\n")
print("-------DECISION TREE--------")
print("Gmean_DT:",Gmean_d)
print("AUC_DT:",Auc_d)
print("f1_DT:",f1_d)
print("precision_DT:",precision_d)
print("recall_DT:",recall_d)

print("mean of Gmean_d:",stat.mean(Gmean_d))
print("mean of AUC_d:",stat.mean(Auc_d))
print("mean of f1_d:",stat.mean(f1_d))
print("mean of precision_d:",stat.mean(precision_d))
print("mean of recall_d:",stat.mean(recall_d))

print("standard deviation of Gmean_d:",stat.stdev(Gmean_d))
print("standard deviation of AUC_d:",stat.stdev(Auc_d))
print("standard deviation of f1_d:",stat.stdev(f1_d))
print("standard deviation of precision_d:",stat.stdev(precision_d))
print("standard deviation of recall_d:",stat.stdev(recall_d))



-------SVM--------
Gmean_SVM: [0.9816529342899867, 1.0, 0.9636610526559151, 0.7803456961553689, 0.8004799921869951]
AUC_SVM: [0.990909090909091, 1.0, 0.9818181818181818, 0.8703703703703703, 0.8900673400673401]
f1_SVM: [0.9908256880733944, 1.0, 0.9821428571428572, 0.8870967741935484, 0.8909090909090909]
precision_SVM: [1.0, 1.0, 0.9649122807017544, 0.7971014492753623, 0.875]
recall_SVM: [0.9818181818181818, 1.0, 1.0, 1.0, 0.9074074074074074]
mean of Gmean_s: 0.9052279350576531
mean of AUC_s: 0.9466329966329966
mean of f1_s: 0.9501948820637782
mean of precision_s: 0.9274027459954234
mean of recall_s: 0.9778451178451179
standard deviation of Gmean_s: 0.1058355554176298
standard deviation of AUC_s: 0.06136380956315361
standard deviation of f1_s: 0.05623224504438233
standard deviation of precision_s: 0.08898337413062866
standard deviation of recall_s: 0.040155238446966494


-------KNN--------
Gmean_KNN: [0.9636610526559151, 1.0, 0.9636610526559151, 0.8335322711253156, 0.8615968921749447]
AU

In [57]:
list=['ecoli0137vs26',stat.mean(Auc_k),stat.stdev(Auc_k),stat.mean(Gmean_k),stat.stdev(Gmean_k),stat.mean(Auc_d),stat.stdev(Auc_d),stat.mean(Gmean_d),stat.stdev(Gmean_d),stat.mean(Auc_s),stat.stdev(Auc_s),stat.mean(Gmean_s),stat.stdev(Gmean_s)]
list

df_excel = pd.read_excel('/content/drive/MyDrive/pso_1.xlsx')
df_excel

,Unnamed: 0,KNN,Unnamed: 2,Unnamed: 3,Unnamed: 4,Decision Tree,Unnamed: 6,Unnamed: 7,Unnamed: 8,SVM,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN,AUC,NaN,Gmean,NaN
1,Dataset,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev,mean,st.dev


In [58]:
df_excel.loc[len(df_excel)]=list
df_excel.to_excel('/content/drive/MyDrive/pso_1.xlsx')